In [ ]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd

from kasearch import SearchOAS, SetQueries

In [ ]:
raw_queries = [
    'QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSS',
    'EVQLQQSGTVLARPGASVKMSCEASGYTFTNYWMHWVKQRPGQGLEWIGAIYPGNSDTSYIQKFKGKAKLTAVTSTTSVYMELSSLTNEDSAVYYCTLYDGYYVFAYWGQGTLVTVSA',
    'QVQLLESGGGLVQPGGSLRLSCAASGFTFSTAAMSWVRQAPGKGLEWVSGISGSGSSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCARELSYLYSGYYFDYWGQGTLVTVSS',
]

In [ ]:
QueryDB = SetQueries(raw_queries, n_jobs=2)
QueryDB.queries[0]

In [4]:
DATABASE_PATH = "/data/icarus/olsen/projects/scan_oas/reports/oas-unpaired/oas-unpaired"

In [5]:
search_oas = SearchOAS(database_path=DATABASE_PATH, 
                                  chain='Heavy', 
                                  allowed_species=['Human'], 
                                  n_jobs=1)

In [6]:
%%timeit -n 1 -r 1
search_oas.search(QueryDB.queries[0], keep_best_n=2)

3min 6s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
search_oas.current_best_identities

array([[0.68947407, 0.76388889, 0.83333333],
       [0.68907563, 0.75714286, 0.79166667]])